In [4]:
from pyspark.sql.functions import lower, regexp_replace, regexp_extract, col, trim, when, instr, lit, split, size, avg, isnan, when, count, isnull, mean, coalesce
from pyspark.sql.functions import concat_ws
from pyspark.sql import Window

special_char = '[^a-z0-9A-Z_ ' \
               'àáãạảăắằẳẵặâấầẩẫậèéẹẻẽêềếểễệđìíĩỉịòóõọỏôốồổỗộơớờởỡợùúũụủưứừửữựỳỵỷỹýÀÁÃẠẢĂẮẰẲẴẶÂẤẦẨẪẬ' \
               'ÈÉẸẺẼÊỀẾỂỄỆĐÌÍĨỈỊÒÓÕỌỎÔỐỒỔỖỘƠỚỜỞỠỢÙÚŨỤỦƯỨỪỬỮỰỲỴỶỸÝ]+'

In [3]:
from pyspark.sql import SparkSession

# Create a DataFrame using SparkSession
spark = (SparkSession
    .builder
    .appName("lazada_cleaning")
    .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/11 10:09:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType, MapType

path = '/home/jazzdung/Downloads/product_lazada.ndjson'
# path = '/home/jazzdung/E-Commerce-Support-System/mr_clean/product.ndjson'

# spark.conf.set("spark.sql.caseSensitive", True)
# df = spark.read.json(path)
# spark.conf.set("spark.sql.caseSensitive", False)

# to convert attrs to String
schema = StructType([
    StructField("product_name", StringType(), True),
    StructField("avg_rating", StringType(), True),
    StructField("price", StringType(), True),
    StructField("brand_name", StringType(), True),
    StructField("num_review", StringType(), True),
    StructField("attrs", MapType(StringType(),StringType(),True)), 
    StructField("category", StringType(), True),
    StructField("shop_info", StringType(), True),
    StructField("product_desc", StringType(), True),
    StructField("url", StringType(), True)
])

df = spark.read.format("json").schema(schema).load(path)


df.count()
df.printSchema()
# df.show(100, truncate=False)

root
 |-- product_name: string (nullable = true)
 |-- avg_rating: string (nullable = true)
 |-- price: string (nullable = true)
 |-- brand_name: string (nullable = true)
 |-- num_review: string (nullable = true)
 |-- attrs: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- category: string (nullable = true)
 |-- shop_info: string (nullable = true)
 |-- product_desc: string (nullable = true)
 |-- url: string (nullable = true)



In [65]:
def clean_product_name(df):
    # Lowercase
    product_name = lower(col('product_name'))

    # # Remove like tier
    # product_name = regexp_replace(product_name, 'yêu thích\n|yêu thích\+\n', ' ')

    # Remove contents inside [], option indicate promotion, prices    
    product_name = regexp_replace(product_name, r'\[.*?\]', ' ')

    # Remove contents inside [], option indicate promotion, prices    
    product_name = regexp_replace(product_name, r'\(.*?\)', ' ')

    # Remove special character
    product_name = regexp_replace(product_name, special_char, ' ')
    # Remove redundant whitespaces  
    product_name = regexp_replace(product_name, ' +', ' ')

    # Trim
    product_name = trim(product_name)
    return df.withColumn('product_name', product_name)

def clean_price(df):
    value = col('price')   
    value = regexp_replace(value, '₫' , '')
    value = regexp_replace(value, ',', '')
    value = value.cast('int')

    return df.withColumn('price', value)

def clean_brand(df):
    brand_name = lower(col('brand_name'))
    brand_name = regexp_replace(brand_name, 'no brand', 'no info')
    brand_name = regexp_replace(brand_name, special_char, ' ')
    brand_name = regexp_replace(brand_name, ' +', ' ')
    brand_name = trim(brand_name)
    return df.withColumn('brand_name', brand_name)

def clean_review(df):
    num_review = lower(col('num_review'))
    num_review = regexp_replace(num_review, 'no ratings', '0')
    num_review = regexp_replace(num_review, ' ratings', '')
    # num_review = regexp_replace(num_review, 'không có đánh giá', '0')
    # num_review = regexp_replace(num_review, ' đánh giá', '')
    num_review = num_review.cast('int')

    return df.withColumn('num_review', num_review)

def clean_attrs(df):
    attrs = lower(col('attrs').cast('string'))
    attrs = regexp_replace(attrs, special_char, ' ')
    attrs = regexp_replace(attrs, ' +', ' ')
    attrs = trim(attrs)
    return df.withColumn('attrs', attrs)

def clean_desc(df):
    product_desc = lower(col('product_desc'))
    product_desc = regexp_replace(product_desc, ' &amp;', ',')
    product_desc = regexp_replace(product_desc, '<svg.*?</svg>|<div>|div|class=|"|<label.*?>|<flex.*?>| href=/', '')
    product_desc = regexp_replace(product_desc, '</a>', '-')
    product_desc = regexp_replace(product_desc, '</label>', ': ')
    product_desc = regexp_replace(product_desc, '< ', '<')
    product_desc = regexp_replace(product_desc, "\/.*?\>","/>")
    product_desc = regexp_replace(product_desc, '<a ', '<')
    product_desc = regexp_replace(product_desc, '<p ', '<')

    product_desc = regexp_replace(product_desc, '</>', ' ')
    product_desc = regexp_replace(product_desc, '<.*?>', ' ')
    product_desc = regexp_replace(product_desc, '\\n', ' ')
    product_desc = regexp_replace(product_desc, special_char, ' ')

    product_desc = regexp_replace(product_desc, ' +', ' ')
    product_desc = trim(product_desc)

    return df.withColumn('product_desc', product_desc)

def extract_first_category(df):
    category = regexp_extract('category', '(.+?)/', 1)
    return df.withColumn('first_category', category)

def extract_second_category(df):
    category = col('category')
    cat_list = split(category, r"/")

    return df.withColumn('second_category', 
        when (
            size(cat_list) > 1,
            concat_ws(' / ',cat_list[0],cat_list[1])
        ).otherwise('no info')
    )

def extract_third_category(df):
    category = col('category')
    cat_list = split(category, r"/")

    return df.withColumn('third_category', 
        when (
            size(cat_list) > 2,
            concat_ws(' / ',cat_list[0],cat_list[1], cat_list[2])
        ).otherwise('no info')
    )

def extract_shop_name(df):
    shop_info = col('shop_info')
    shop_name = regexp_extract(shop_info, '\\n(.+?)\\n', 1)
    return df.withColumn('shop_name', shop_name)

def extract_shop_rating(df):
    shop_info = col('shop_info')
    shop_rating = regexp_extract(shop_info, 'Seller Ratings\\n(.+?)\\n', 1)
    shop_rating = regexp_replace(shop_rating, '%', '')
    shop_rating = shop_rating.cast('float') / 100
    return df.withColumn('shop_rating', shop_rating)

def extract_ship_on_time(df):
    shop_info = col('shop_info')
    ship_on_time = regexp_extract(shop_info, 'Ship On Time\\n(.+?)\\n', 1)
    ship_on_time = regexp_replace(ship_on_time, '%', '')
    ship_on_time = ship_on_time.cast('float') / 100
    return df.withColumn('ship_on_time', ship_on_time)

def extract_shop_reply_percectage(df):
    shop_info = col('shop_info')
    shop_reply_percectage = regexp_extract(shop_info, 'Chat Response\\n(.+?)\\n', 1)
    shop_reply_percectage = regexp_replace(shop_reply_percectage, '%', '')
    shop_reply_percectage = shop_reply_percectage.cast('float') / 100
    return df.withColumn('shop_reply_percectage', shop_reply_percectage)

# Implement

In [66]:
# Clean
df_cleaned = clean_product_name(df)
df_cleaned = clean_price(df_cleaned)
df_cleaned = clean_brand(df_cleaned)
df_cleaned = clean_review(df_cleaned)
df_cleaned = clean_attrs(df_cleaned)
df_cleaned = clean_desc(df_cleaned)
df_cleaned = extract_first_category(df_cleaned)
df_cleaned = extract_second_category(df_cleaned)
df_cleaned = extract_third_category(df_cleaned)
df_cleaned = extract_shop_name(df_cleaned)
df_cleaned = extract_shop_rating(df_cleaned)
df_cleaned = extract_ship_on_time(df_cleaned)
df_cleaned = extract_shop_reply_percectage(df_cleaned)

# Cast
df_cleaned = df_cleaned.withColumn("avg_rating",df_cleaned["avg_rating"].cast('double'))

# Rename
df_cleaned = df_cleaned.withColumnRenamed('brand_name', 'brand')

# Drop
df_cleaned = df_cleaned.drop("shop_info")
df_cleaned = df_cleaned.drop("category")

df_cleaned.printSchema()
df_cleaned.show(100, truncate=False)

root
 |-- product_name: string (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- price: integer (nullable = true)
 |-- brand: string (nullable = true)
 |-- num_review: integer (nullable = true)
 |-- attrs: string (nullable = true)
 |-- category: string (nullable = true)
 |-- description: string (nullable = true)
 |-- url: string (nullable = true)
 |-- first_category: string (nullable = true)
 |-- second_category: string (nullable = false)
 |-- third_category: string (nullable = false)
 |-- shop_name: string (nullable = true)
 |-- shop_rating: double (nullable = true)
 |-- ship_on_time: double (nullable = true)
 |-- shop_reply_percectage: double (nullable = true)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+------+-------+----------+----------------------------------+----------------------------------------------------